TODO
--
* [X] Cómo cargar archivo en layout
* [ ] Subir instancias al repositorio
* [ ] Algoritmo general que seleccione stack usando heurística, desmantela lo necesario y vuelve a llenar. Usar funciones `SDpp` y `SFpp` (se pueden modificar)
* [ ] Arreglar rank y se genere usando contenedores que se van a extraer y para que acepte repetidos 
* [ ] Probar usando instancias y revisar *posibles errores*

Cosos del profe para tomar como referencia
--

In [1]:
import Layout
import Greedy

def get_ranks(stack):
    r=1
    rank = {}
    for i in sorted(stack, reverse=True):
        rank[i] = r
        r += 1
    return rank

def fill_stack(layout, s_d, n, ori, rank):
    for i in range(n):
        s_o = Layout.select_origin_stack(layout, s_d, ori, rank)
        layout.move(s_o,s_d)

import numpy as np
def capacity(layout, s_o):
    capacity = 0
    for i in range(len(layout.stacks)):
        if i==s_o: continue
        capacity += layout.H - len(layout.stacks[i])
    return capacity            

def force_move(layout, s_o, pos, s_d): #pixie
    while pos<-1:
        s_tmp = Layout.select_destination_stack(layout, s_o, black_list=[s_d])
        layout.move(s_o,s_tmp)
        pos += 1
    return layout.move(s_o,s_d)

def search_highest(layout, c, ub, s_d):
    ret = None
    for pos in range(-1,-10,-1):
        max_c = 0
        for s in range(len(layout.stacks)):
            if s==s_d: continue
            ss = layout.stacks[s]
            if len(ss) < -pos: continue
            if ss[pos]>=c and ss[pos]<=ub and ss[pos]>max_c: 
                max_c=ss[pos]
                ret= (s,pos)
        if ret != None: return ret
    return None

Funciones mágicas pero del profe
--

In [2]:
def SDpp(layout, s_o, rank):
    capac = capacity(layout,s_o) # espacio libre
    ss_o = layout.stacks[s_o]
    while len(layout.stacks[s_o])>0:
        top = Layout.gvalue(ss_o)
        slack = capac-len(ss_o) # holgura
        s_d = Layout.select_destination_stack (layout, s_o, max_pos=rank[top]+slack)
        pos = layout.H - len(layout.stacks[s_d])
        if rank[top] < pos - slack:  # rellenar stack s_d
            #c=layout.stacks[s_o].pop(-1)
            fill_stack(layout, s_d, (pos - slack) - rank[top], s_o, rank)
            #layout.stacks[s_o].append(c)

        capac -= 1
        layout.move(s_o,s_d)
        
def SFpp(layout, s_d, rank, n=10):
    ub = 100
    cont = 0
    for r in rank:
        while True:
            (s, pos) = search_highest(layout, r, ub, s_d)
            if layout.sorted_elements[s] > len(layout.stacks[s])+pos: break #element is sorted
            c=force_move(layout, s, pos, s_d)
            ub=r
            cont+=1
            if cont == n: return
            if r==c: break
    while cont < n:
        Greedy.SF_move_d(layout, s_d)
        cont += 1
####
def SDppp(layout, s_o, rank):
    capac = capacity(layout,s_o) # espacio libre
    ss_o = layout.stacks[s_o]
    
    sort = sorted(ss_o)
    sort.reverse()
    copia = ss_o.copy()
    
    re=0
    for i in (range(len(sort))):
        if sort[i] != copia[i]:
            re=i
            break;
    
    #Se quitan 
    while len(layout.stacks[s_o])>re:
        top = Layout.gvalue(ss_o)
        slack = capac-len(ss_o) # holgura
        s_d = Layout.select_destination_stack (layout, s_o, max_pos=rank[top]+slack)
        pos = layout.H - len(layout.stacks[s_d])
        if rank[top] < pos - slack:  # rellenar stack s_d
            #c=layout.stacks[s_o].pop(-1)
            fill_stack(layout, s_d, (pos - slack) - rank[top], s_o, rank)
            #layout.stacks[s_o].append(c)

        capac -= 1
        layout.move(s_o,s_d)

def SFppp(layout, s_d, rank, n):
    ub = 100
    cont = 0
    for r in rank:
        while True:
            (s, pos) = search_highest(layout, r, ub, s)
            if layout.sorted_elements[s] >= len(layout.stacks[s])+pos: break #element is sorted
            c=force_move(layout, s, pos, s_d)
            ub=r
            cont+=1
            if cont == n: return
            if r==c: break
    while cont < n:
        Greedy.SF_move_d(layout, s_d)
        cont += 1

Algunos problemas conocidos (de las funciones de arriba)
----

- `SDpp` desmantela completamente el stack [CHECK]
- `SFpp` tiene errores
- Puede que tenga problemas si hay elementos con el mismo valor (rank es un mapa por lo que considerará uno de los elementos)
- Si alguna función no puede hacer su trabajo fallará
- El tema de la holgura (slack) hay que revisarlo. Es para instancias donde hay *espacio de sobra* para colocar elmentos

Y aqui va lo mio supongo :p
----
<a rel="pixie">![Foo](https://images-na.ssl-images-amazon.com/images/I/61mBskokGZL._AC_SL1280_.jpg)</a>

Cosas mias que no se porque funcionan pero funcionan
--

* La función `read(path)` recibe como parametro `path` y retorna un layout.
* La función `select_col(layout)` selecciona una columna para trabajar (la que se re-estructurara) y omite las columnas ordenadas, retorna el indice de la columna seleccionada o un -1 en caso de que todo este ordenado.
* La función `capac(layout,column)` a.k.a lo que hice en java pero aca para ver el espacio disponible dinamico.
* La funcion `remove_until(column)` indica cuantos elementos hay que sacar de la columna para reordenar, en caso que algunos contenedores ya se encuentren ordenados en la base del stack.


Lectura de archivo
--


In [3]:



flag = 0
#Columnas
columns = 0
#Total contenedores
total = 0
maxSize = 0
stacks = []
with open("C:\\Users\\Leviathan\\Desktop\\trabajo\\Referencias\\BF\\test\\10-6\\data10-6-17.dat") as file:
    for line in file:
        s = line.split()
        #Si es la primera linea entonces se guarda las columnas y el total
        if flag == 0:
            #Actualiza columns
            columns = s[0]
            #Actualiza total
            total = s[1]
            #Actualiza flag para que no pase denuevo
            flag=1
        #En caso contrario sen otros datos de interes
        else:
            #Se ve el size mayor de la columna para definirlo automatico
            aux=int(s[0])
            if maxSize <= aux:
                maxSize = aux;
            
            #Se crean los stacks
            temporalStack = []
            for i in range(1,len(s)):
                temporalStack.append(int(s[i]))
            stacks.append(temporalStack)

#Una vez leido el archivo imprimo los stacks para ver si quedo bien
#for s in stacks:
#  print (s)
#maxSize += 2
#print (maxSize)



In [4]:
#Se crea el layout
maxSize += 2
layout = Layout.Layout(stacks, maxSize)
#move c_o->c_d
#layout.move(0,1)

print (maxSize)





12


Lectura pero en funcion
--


In [5]:
def read(path):
    flag = 0
    #Columnas
    columns = 0
    #Total contenedores
    total = 0
    maxSize = 0
    stacks = []
    with open("C:\\Users\\Leviathan\\Desktop\\trabajo\\Referencias\\BF\\test\\10-6\\data10-6-17.dat") as file:
        for line in file:
            s = line.split()
            #Si es la primera linea entonces se guarda las columnas y el total
            if flag == 0:
                #Actualiza columns
                columns = s[0]
                #Actualiza total
                total = s[1]
                #Actualiza flag para que no pase denuevo
                flag=1
            #En caso contrario sen otros datos de interes
            else:
                #Se ve el size mayor de la columna para definirlo automatico
                aux=int(s[0])
                if maxSize <= aux:
                    maxSize = aux;

                #Se crean los stacks
                temporalStack = []
                for i in range(1,len(s)):
                    temporalStack.append(int(s[i]))
                stacks.append(temporalStack)
    #Se crea el layout
    maxSize += 2
    layout = Layout.Layout(stacks, maxSize)
    
    return layout

#Test
path="C:\\Users\\Leviathan\\Desktop\\trabajo\\Referencias\\BF\\test\\10-6\\data10-6-17.dat"
layout=read(path)
print (layout.stacks)
print (layout.H)

[[6, 3, 56, 2, 33, 59, 51, 25, 29, 44], [24, 46, 23, 37, 42, 38, 30, 28, 19, 32], [11, 60, 26, 5, 31, 43, 58, 20, 13, 40], [55, 41, 21, 34, 14, 17, 45, 9, 36, 12], [4, 16, 8, 27, 52, 15, 49, 7, 47, 1], [39, 48, 18, 35, 54, 57, 50, 22, 53, 10]]
12


In [6]:
#Por conveniencias de la vida, tendre un field directo  a los stacks de layout
field=layout.stacks  
#print(field)

import copy 

#Se revisa una columna posible que nos generara la menor cantidad de problemas para ordenar
def select_col(layout):

    field=layout.stacks  
    best=0
    col=-1
    
    for i in field:
        
        sort=copy.deepcopy(i)
        sort.sort(reverse=True)
        copia=copy.deepcopy(i)
        
        #Esto es para que no tome las columnas ordenadas
        if sort != copia:
            #Mientras no sean iguales
            while sort != copia:
                #Quitamos el superior
                sort.pop(len(sort)-1)
                copia.pop(len(copia)-1)

                if sort == copia:
                    #Se revisa que sea mejor que el anterior
                    if best < len(copia):
                        best=len(copia)
                        col=i
                    break
    
    for i in range(0,len(field)-1):
        if layout.stacks[i] == col:
            return i
    #En caso que no queden columnas
    return -1

seleccionada=select_col(layout)
print (seleccionada)
    

3


In [7]:
#Una vez esta seleccionada la columna se trabaja con ella, primero vemos la capacidad que tiene cada columna por separado
def capac(layout,column):
    field=layout.stacks
    capacidad=[]
    contador=0
    for i in field:
        if contador != column:
            capacidad.append(layout.H-len(i))
        else:
            capacidad.append(0)
        contador+=1
    return capacidad

print(seleccionada)
capacidad=capac(layout,seleccionada)
#print (field)
print (capacidad)


3
[2, 2, 2, 0, 2, 2]


In [8]:
#Se crea el field falso
stacks=[]

for i in range(0,len(capacidad)):
    stacks.append([])
print (stacks)

#Se crea donde guardar los movimientos (terminara siendo una lista de pares) seria un move[0] to move[1], move[2] to move[3] y asi
movs=[]




[[], [], [], [], [], []]


In [49]:
#Finalmente vemos hasta donde se debe sacar de la columna que queremos arreglar
columna=field[select_col(layout)]

#Retorna la cantidad de veces que debe sacarse un contenedor de la parte superior de la columna
def remove_until(column):
    sort=copy.deepcopy(column)
    sort.sort(reverse=True)
    copia=copy.deepcopy(column)
    
    contador=1
    while sort != copia:
        sort.pop(len(sort)-1)
        copia.pop(len(copia)-1)
        
        if sort == copia:
            return contador
        
        contador+=1

#test funcion
print (remove_until(columna))
print (columna)

9
[55, 41, 21, 34, 14, 17, 45, 9, 36, 12]


In [50]:
#Recopilacion de los datos importantes
seleccionada=select_col(layout)
#Capacidad maxima
capacidad=capac(layout,seleccionada)
#Columna
columna=copy.deepcopy(layout.stacks[seleccionada])
#Contenedores a retirar
retirar=remove_until(columna)
#Movimientos
movs=[]
stacks=[]
for i in range(0,len(capacidad)):
    stacks.append([])
#~°~°~°~°~°~°~°~°~°~~°~~°~~°~~°~REVISION SI TOMO LOS DATOS BIEN~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~
#~°~°~°~°~°~°~°~°~°~~°~~°~~°~~°~SEPARADOR MARCA FRUNA~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~
print(seleccionada)
print(retirar)
print(capacidad)
print(columna)
print(stacks)

#~°~°~°~°~°~°~°~°~°~~°~~°~~°~~°~SEPARADOR MARCA FRUNA~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~
#~°~°~°~°~°~°~°~°~°~~°~~°~~°~~°~FIN REVISION~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~°~

3
9
[2, 2, 2, 0, 2, 2]
[55, 41, 21, 34, 14, 17, 45, 9, 36, 12]
[[], [], [], [], [], []]


In [51]:
#Defino una clase conveniente
class bloque():
    
    def  __init__ (self, prioridad,vector):
        self.prioridad=prioridad
        self.menores = [] 
        for i in vector:
            if prioridad >= i:
                self.menores.append(i)
        self.menores.sort(reverse = True)

                
def putIn(stacks,capacidad,contenedor):
    
    posibles=[]
    posibles_vacios=[]
    for i in range(len(stacks)):
        #En caso que exista espacio
        if capacidad[i]>0:
            temp=stacks[i]
            #print(len(temp)-1)
            if not temp:
                posibles_vacios.append(i)
            else:
                bloque=temp[len(temp)-1]
                #Si contiene al bloque dentro de los menores
                if bloque.prioridad in contenedor.menores:
                    posibles.append(i)
                    
            #print(bloque.prioridad)
    #print(posibles)
    empty=-1
    bestcol=-1
    mayor=0
    #Si existen posibles que no sean vacios
    if len(posibles) > 0:
        #Revisar cual es el mejor (veamos que pasa si siempre se va por el mayor)
        for i in posibles:
            temp=stacks[i]
            bloque=temp[len(temp)-1]
            
            if bloque.prioridad > mayor:
                mayor=bloque.prioridad
                bestcol=i
        #Retorna la mejor columna de momento
        return bestcol
    else:
        #Retornar el primer vacio, total no es como que importe realmente en que vacio se queda
        if len(posibles_vacios) > 0:
            
            for i in posibles_vacios:
                return i
        
        else:
            #El peor caso, esto significa que gg wp pa la casa y hay que reordenar pero como aun no tengo la funcion 
            #lo dejo en un funcion="pendiente"
            funcion="pendiente"

#DELETE THIS         
def reordenar(stacks,capacidad,contenedor):
    #Stacks tiene el nuevo field
    #Capacidad tiene la capacidad de cada stack
    #Contenedor tiene los datos de contenedor
    
    #Se ve el mayor
    mayor=0
    col=-1
    cont=0
    for i in stacks:
        if len(i) > 0:
            bloque=temp[len(temp)-1]
            
            if bloque.prioridad > mayor:
                mayor=bloque.prioridad
                col=cont
        cont+=1
            
def pixieOrder(stacks,capacidad):
    #Se recorre el stack para encontrar todos los actuales
    contenedores=[]
    for i in stacks:
        #En caso que no este vacio recorro todo el stack
        if len(i) > 0:
            for j in i:
                contenedores.append(j)
    #Se ordenan
    contenedores.sort(reverse=True)
    #aqui se ordenan las cosas
    
                

In [52]:
#A quitar cosas
vector=copy.deepcopy(layout.stacks[seleccionada])
while retirar>0:
    contenedor=columna.pop(len(columna)-1)
    print(contenedor)
    reordenar(stacks,capacidad,contenedor)
    #Una vez sacado el contenedor se coloca donde se pueda
    coso=bloque(contenedor,vector)
    movimiento=putIn(stacks,capacidad,coso)
    #print(coso.menores)
    retirar-=1


12
[]
[]
[]
[]
[]
[]
36
[]
[]
[]
[]
[]
[]
9
[]
[]
[]
[]
[]
[]
45
[]
[]
[]
[]
[]
[]
17
[]
[]
[]
[]
[]
[]
14
[]
[]
[]
[]
[]
[]
34
[]
[]
[]
[]
[]
[]
21
[]
[]
[]
[]
[]
[]
41
[]
[]
[]
[]
[]
[]
